In [71]:
import dash

In [73]:
from dash import dcc
from dash import html
from dash.dependencies import Input,Output
import plotly.express as px
import pandas as pd

In [75]:
spacex_df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")

In [77]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [79]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [93]:

app=dash.Dash(__name__)
d_options=[ {'label':'All Sites','value':'ALL'}]
for site in spacex_df['Launch Site'].unique():
    d_options.append({'label':site,'value':site})
    
app.layout=html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',style={'textAlign':'center','color':'#503D36','font-size':40}),
    dcc.Dropdown(id='site-dropdown',options=d_options,searchable=True,value='ALL',placeholder='Select a Launch Site here'),
    html.Br(),
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(id='payload-slider',
                   min=0,max=10000,step=1000,marks={0:'0',2500:'2500',5000:'5000',7000:'7500',10000:'10000'},value=[min_payload,max_payload]
                   ),
    html.Div([dcc.Graph(id='success-payload-scatter-chart')])
])
@app.callback(
    Output(component_id='success-pie-chart',component_property='figure'),
    Input(component_id='site-dropdown',component_property='value')
)
def update_pie(launch_site):
    if(launch_site=='ALL'):
        fig=px.pie(spacex_df,values='class',names='Launch Site',title='Total Success launches By each Site')
    else:
        f_df=spacex_df[spacex_df['Launch Site']==launch_site]
        fig=px.pie(f_df,names='class',title=f"total succesful launches for {launch_site}",color_discrete_map={0:'red',1:'green'},color='class')
    return fig

@app.callback(
    Output('success-payload-scatter-chart','figure'),
    [Input('site-dropdown','value'),
    Input('payload-slider','value')]
)

def update_scatter(entered_site,payload_range):
    low,high=payload_range
    p_range=(spacex_df['Payload Mass (kg)']>=low) & (spacex_df['Payload Mass (kg)']<=high)
    if entered_site=='ALL':
        f_df=spacex_df[p_range]
        fig=px.scatter(f_df,x='Payload Mass (kg)',y='class',
                       color='Booster Version Category',
                       title='Correlation between Payload and Success for all Sites'
                       
                      )
    else:
        f_df=spacex_df[(spacex_df['Launch Site']==entered_site) & p_range]
        fig=px.scatter(
            f_df,
            x='Payload Mass (kg)',
            y='class',
            color='Booster Version Category',
            title=f"Correlation between Payload and Success for {entered_site}"

        )
    return fig
        
        
if __name__ == '__main__':
    app.run()